# **LAB THREE: SELECTIVE EXCITATION**

This lab covers selective excitation, RF pulse design using Sinc and SLR, slice profiles, slice offsetting, and the relationships between pulse bandwidth, gradient strength, slice thickness and SNR.

-------------------------------------------------------------------------------------------------------------------------------------------------------
 #### **Required Resources**
 
 For this lab you will need:
 - ilumr fitted with 10mm RF Probe
 - Shim Sample (#001)
 - Phantom #005 (Three Cell Phantom)
 - Phantom #006
 
 -------------------------------------------------------------------------------------------------------------------------------------------------------

>-------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Setup Task: Run the Notebook**
> 
> 1. Edit the cell below to set the `LAB_USER_NAME` variable to your name
> 2. Click **Run->Run All Cells** in the in top menu bar of jupyterlab
> 3. Open the Table of Contents side-bar on the left edge of jupyterlab to aid in navigation
> 
> -------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
LAB_USER_NAME = 'REPLACE_ME'

**Important**: To initialise this notebook, edit the cell above to set `LAB_USER_NAME` to your name, then click **Run->Run All Cells** in the top menu bar.

In [ ]:
import panel as pn
import sys
import os
import yaml
import numpy as np

from matipo import Sequence, SEQUENCE_DIR, GLOBALS_DIR, DATA_DIR, Unit
from matipo.util.decimation import decimate
from matipo.util.fft import fft_reconstruction
from matipo.util.etl import deinterlace

from matipo.experiment.base_experiment import BaseExperiment, auto_inputs, PlotInterface
from matipo.experiment.models import PLOT_COLORS, SITickFormatter, SIHoverFormatter

from matipo.experiment.plots import SignalPlot, SpectrumPlot, ImagePlot, Image1DPlot, ComplexDataLinePlot

pn.extension(inline=True)

WORKSPACE = os.path.join(LAB_USER_NAME, 'Lab 3')
LAB_DIR = os.path.join(DATA_DIR, WORKSPACE)
os.makedirs(LAB_DIR, exist_ok=True)
print('Data will be saved to', LAB_DIR)

GAMMA_BAR = 42.58e6
RESOLUTION = 64
FOV = 12e-3 # m
AXIS_VECTORS = {
    'X': np.array([1,0,0], dtype=float),
    'Y': np.array([0,1,0], dtype=float),
    'Z': np.array([0,0,1], dtype=float)
}

try:
    with open(GLOBALS_DIR+'hardpulse_90.yaml', 'r') as f:
        data = yaml.load(f, Loader=yaml.SafeLoader)
        AREA_90 = data['a_90']*data['t_90']
except IOError:
    print('Unable to load pulse calibration, using default value')
    AREA_90 = 0.3*32e-6
    
def estimate_a_90(pulse_shape, pulse_width):
    shape_area = np.mean(pulse_shape)*pulse_width
    return min(1, abs(AREA_90/shape_area))

## 1. Projection Images
Thus far, the 2D images created in the labs have been projection images. These images are created using a rectangular RF-pulse (known as a hard pulse) and both frequency and phase encoding gradients (Fig.1.1). The frequency and phase encoding gradients allow the aquired signals to be sorted into 2D space to produce an image. The orientation of the image will depend on which axes the frequency and phase gradients were applied along. Since there is no form of encoding in the third dimension of the sample, the signals along this axis will be summed in the 2D image.   

<center><img src="Images/projection.png" width="1300"></center>
<center><figcaption style="width: 600px;">Figure 1.1: Setting the frequency and phase encoding in the x and y axes produces a projection image where the varying signal components in the z direction are summed into average values.  </figcaption></center>


> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Task 1.1: Generating a Projection Image**
> 1. Insert **Phantom #005** at the correct depth.
> 2. Choose XY for the phase encoding and frequency encoding axes and start the experiment by pressing "Run". (If the image is offset, run the Find Frequency dashboard)
> 3. **Question:** Can you tell the vertical position of the cells?
> 4. Try changing one of the encoding axes to Z to get a side profile. If the sample is off-centre, use this image to adjust it.
> 5. Insert **Phantom #006** and repeat the XY and ZY images.
> 6. **Question:** Is the text embedded in the phantom visible/readable?
> -------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
from base_2DRARE_experiment import Base2DRAREExperiment

# Experiment 1: 2D RARE projection image experiment with selectable phase and frequency encoding axis

class Projection2DRAREExperiment(Base2DRAREExperiment):
    def setup(self):
        super().setup() # call parent class setup method
        
        self.title = "2D Projection Experiment"
        
        self.inputs = {
            'axis_f': pn.widgets.RadioButtonGroup(name='Freq Enc Axis', options=['X', 'Y', 'Z'], value='X', width = 200),
            'axis_p': pn.widgets.RadioButtonGroup(name='Phase Enc Axis', options=['X', 'Y', 'Z'], value='Y', width = 200)
        }
        
        self.FOV = FOV

    def layout_inputs(self):
        return pn.Column(
            pn.Row(pn.pane.Markdown('### Frequency Encoding Axis', width=200), self.inputs['axis_f']),
            pn.Row(pn.pane.Markdown('### Phase Encoding Axis', width=200), self.inputs['axis_p']),
            sizing_mode='stretch_width'
        )

    def update_par(self):
        self.seq.setpar(
            n_scans=16,
            t_end=1,
            t_dw=20e-6/self.DEC,
            n_samples=RESOLUTION*self.DEC,
            n_phase_2=RESOLUTION,
            # set gradient directions
            g_read=AXIS_VECTORS[self.inputs['axis_f'].value],
            g_phase_2=AXIS_VECTORS[self.inputs['axis_p'].value]
        )
        super().update_par() # automatically determines gradient magnitudes based on the self.FOV attribute

exp1 = Projection2DRAREExperiment(state_id='exp1')
exp1()

As you can see, projection images aren't always useful because summing a large amount of signal in the object's third dimension makes it almost impossible to see details that are only present in specific parts of the sample. If we want to see detail in a specific portion of the object we need to be able to limit the amount of signal that is being summed in one direction. This is done through slice selection.

## 2. Frequency Selective Excitation

In order to image a single slice rather than the entire sample, two main changes need to be made to the pulse sequence:

1) A slice-select gradient must be added along the axis perpendicular to the plane of the desired slice. This causes the spins along the chosen axis to precess at different frequencies making it possible to selectively excite them by transmitting at a specific frequency.

2) The RF excitation pulse must be modified to only transmit a small range of frequencies. So far, we have used short (~20 us) high amplitude pulses which excite a very broad range of frequencies (called a "hard pulse"). As the pulse width is increased, the range of frequencies contained in it (the bandwidth) decreases, so we need to increase the pulse length to achieve a narrow slice. Flip angle is proportional to the area of the pulse, so a long, narrow-bandwidth pulse needs less amplitude to achieve the same flip angle. This is called a "soft pulse".

The resulting pulse sequence is shown in Figure 2.1 below. 

<center><img src="Images/slice.png" width="1000"></center>
<center><figcaption style="width: 600px;">Figure 2.1: Slicing along the z axis allows the sample to be imaged at a specific cross section index     </figcaption></center>


The relationship of the slice thickness ($\Delta$z) to bandwidth ($\Delta$F) and slice-select gradient strength ($G_{ss}$) is governed by the following equation:

$$\Delta z = \frac{\Delta F}{\bar\gamma \cdot G_{ss}} \tag{1}$$

where $\bar\gamma$ is the gyromagnetic ratio (for hydrogen nucleus $\bar\gamma = 42.58 \times 10^{6} \ \mathrm{Hz}/\mathrm{T}$). Figure 2.2 shows examples of how slice thickness can be controlled by varying $\Delta F$ or $G_{ss}$.

<center><img src="Images/slice_thickness.png" width="800"></center>
<center><figcaption style="width: 800px;">Figure 2.2: How the slice thickness is related to the RF pulse bandwidth and slice-select gradient strength </figcaption></center>

In the next experiment, we will use a simplified pulse sequence (Fig.2.3 below) that performs a spin echo with a constant gradient present during both the RF excitation pulse and the readout. This allows the sample to be selectively excited and then imaged along the same axis to directly measure the excitation profile.

<center><img src="Images/const_grad.png" width=600"></center>
<center><figcaption style="width: 600px;">Figure 2.3: Pulse Sequence for the Constant Gradient Spin Echo experiment</figcaption></center>

> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Task 2.1:**
> 1. Insert the **Shim Sample (#001)** and run the below experiment with a pulse width of 20 us. You will get a 1D Z axis projection image of the sample.
> 2. Increase the pulse width and rerun the experiment. Try with various pulse widths up to 1000 us.
> 3. Note: With a short RF pulse, there should be roughly equal signal from all parts of the sample along the Z axis.
> 4. **Question:** With a long RF pulse, do you still get equal signal from the whole sample? If not, which region dominates?
> 5. **Question:** How does the Z thickness of the dominant region (slice thickness) vary with the pulse width?
> -------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# Experiment 2
class Lab3Task2SpinEchoExperiment(BaseExperiment):
    
    def setup(self):
        self.title = "Constant Gradient Spin Echo Experiment"
        self.workspace = WORKSPACE
        self.seq = Sequence('programs/SE_const_grad.py')
        self.plots = {
            'signal': SignalPlot(),
            'image': Image1DPlot()
        }
        
        with open(os.path.join(GLOBALS_DIR, 'gradient_calibration.yaml'), 'r') as f:
            data = yaml.load(f, Loader=yaml.SafeLoader)
            self.G_CAL = 1/(GAMMA_BAR*data['gradient_calibration']) # convert
            self.log.debug(f'G_CAL: {self.G_CAL}')
        
        self.inputs['pulse_width'] = pn.widgets.FloatInput(name="Pulse Width (μs)", start=20, end=1000, step=20, value=20, width=80)
        self.DEC = 4
    
    def update_par(self):
        self.seq.setpar(
            a_90=0.5*AREA_90/(self.inputs['pulse_width'].value*1e-6),
            t_90=self.inputs['pulse_width'].value*1e-6,
            g_read=(0,0,0.05/self.G_CAL),
            n_scans=4,
            n_samples=200*self.DEC,
            t_dw=40e-6/self.DEC, # using a long dwell time for narrow bandwith to more easily see the spectrum shape
            t_end=0.5,
            t_echo = 10e-3
        )
    
    async def update_plots(self, *args, **kwargs):
        await self.seq.fetch_data()
        self.plots['signal'].update(seqdata=self.seq.data, t_dw=self.seq.par.t_dw)
        self.plots['image'].update(
            seqdata=self.seq.data,
            t_dw=self.seq.par.t_dw,
            g_read_mag=self.G_CAL*np.linalg.norm(self.seq.par.g_read),
            dec=self.DEC
        )

lab3exp2 = Lab3Task2SpinEchoExperiment(state_id='exp2')
lab3exp2()

> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Task 2.2: 2D RARE Using Rectangular Soft Pulse**
> 1. Insert **Phantom #005**, and run the experiment with the pulse width set to 20 us and the axes set to X-Y to get a projection image.
> 2. Try increasing the pulse width to selectively excite only the middle cell. You can also switch one of the axes to Z to visualise the excitation profile as in the previous experiment.
> 3. **Question**: Approximately how long does the RF pulse need to be to only get noticable signal from one cell?
> 4. **Question**: What is the thickness of this slice? (You can check with the Constant Gradient Spin Echo experiment above)
> 5. **Question**: If you continue increasing the pulse width, how is the SNR of the image related to the slice thickness?
> -------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
class Task3RARE2DExperiment(Base2DRAREExperiment):
    def setup(self):
        super().setup() # call parent class setup method
        
        self.seq = Sequence('programs/custom_pulse_RARE.py')
        
        self.title = "2D Image with Rectangular RF Pulse and Slice Select Gradient"
        
        self.inputs = {
            'axis_f': pn.widgets.RadioButtonGroup(name='Freq Enc Axis', options=['X', 'Y', 'Z'], value='X', width = 200),
            'axis_p': pn.widgets.RadioButtonGroup(name='Phase Enc Axis', options=['X', 'Y', 'Z'], value='Y', width = 200),
            'pulse_width': pn.widgets.FloatInput(name="Pulse Width (μs)", start=20, end=1000, step=20, value=20, width=80)
        }

    def layout_inputs(self):
        return pn.Column(
            self.inputs['pulse_width'],
            pn.Row(pn.pane.Markdown('### Frequency Encoding Axis', width=200), self.inputs['axis_f']),
            pn.Row(pn.pane.Markdown('### Phase Encoding Axis', width=200), self.inputs['axis_p']),
            sizing_mode='stretch_width'
        )
    
    def g_phase_mag(self, fov, gamma_bar=GAMMA_BAR):
        """ Returns phase gradient required for a given FOV """
        dt = self.seq.par.t_phase/(len(self.seq.par.g_phase_2)//2)
        return max(0, min(1, 1/(self.G_CAL*fov*gamma_bar*dt))) # clip to range 0 to 1
    
    def fov_phase(self, gamma_bar=GAMMA_BAR):
        """ Returns phase encode direction FOV """
        dt = self.seq.par.t_phase/(len(self.seq.par.g_phase_2)//2)
        g = self.G_CAL*np.max(np.linalg.norm(self.seq.par.g_phase_2, axis=1))
        return 1/(g*gamma_bar*dt)

    def update_par(self):
        self.seq.setpar(
            a_90=1.0*AREA_90/(self.inputs['pulse_width'].value*1e-6),
            t_90=self.inputs['pulse_width'].value*1e-6,
            shape_90=[1],
            g_slice=(0,0,50e-3/self.G_CAL),
            g_phase_2=np.zeros((RESOLUTION, 3)),
            n_scans=16,
            n_samples=RESOLUTION*self.DEC,
            t_dw=20e-6/self.DEC,
            t_end=1,
            # t_grad_ramp=50e-6, # TODO: fix strange artifacts with larger t_grad_ramp
            # n_grad_ramp=5
        )
        
        # set calculated pars
        self.seq.setpar(
            n_ETL=len(self.seq.par.g_phase_2),
            t_read=self.seq.par.t_dw*self.seq.par.n_samples
        )
        self.seq.setpar(
            t_phase=self.seq.par.t_read/2
        )
        
        g_phase_prop = np.flip(np.linspace(1, -1, RESOLUTION, endpoint=False))
        self.seq.setpar(
            g_read = self.g_read_mag(self.FOV)*AXIS_VECTORS[self.inputs['axis_f'].value],
            g_phase_2 = np.outer(self.g_phase_mag(self.FOV)*g_phase_prop, AXIS_VECTORS[self.inputs['axis_p'].value])
        )
        

lab3exp3 = Task3RARE2DExperiment(state_id='exp3')
lab3exp3()

## 3. Soft Pulse Design

In practice, a slice will have some finite thickness. Choosing a slice thickness is a trade off between two main factors: A thicker slice will have more signal because there are more nuclear spins available for excitation; a thinner slice will have less averaging over the slice axis resulting in better selection of individual features in the sample. In the previous experiments, a rectangular RF pulse with a constant amplitude was used. This resulted in a Sinc function shaped excitation profile (or slice profile). The side-lobes of the slice profile partially excite the top and bottom cells causing them to show faintly in the image. 

For small tip angles ($\alpha \ll 90^\circ$), the small-tip-angle approximation states that the signal spectrum will be approximately proportional to the spectrum of the RF pulse, and therefore will have the same shape. If the tip angle at any point in the sample approaches or exceeds $90^\circ$ this approximation breaks down and the signal spectrum will be distorted, compared to the RF pulse spectrum, due to the sine relationship between signal and tip angle (Fig. 3.1).

<center><img src="Images/tip_angle.png" width="900"></center>
<center><figcaption style="width: 500px;">Figure 3.1: How the RF pulse shape relates to the NMR signal</figcaption></center>

To perfectly isolate the middle cell, it should be fully excited (90 degree tip angle throughout) while the outer cells are not excited at all (0 degree tip angle). This requires the frequency spectrum of the RF pulse to have a rectangular shape, which can be theoretically achieved with a Sinc shaped RF pulse amplitude modulation as shown in Figure 3.2:

<center><img src="Images/soft_pulse_parameters.png" width="600"></center>
<center><figcaption style="width: 600px;">Figure 3.2: An ideal RF pulse creates a rectangular slice select profile</figcaption></center>

In practice, the theoretical Sinc function is infinitely long, and must be truncated to be used. This may be done at the zero crossings to minimise ringing artifacts (Gibbs phenomenon). Additionally, the Sinc shape can be multiplied with a gaussian to further reduce the Gibbs phenomenon. This process is called Apodization.

> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Task 3.1: Visualising Sinc Pulse Design**
>The experiment below is designed to help you visualise how changing the parameters of a soft pulse changes the profile in the frequency domain. The RF pulse you design will be used for the following experiments.
> 1. Experiment with the three soft pulse parameters and observe how the shape of the RF-pulse and slice profile changes. 
> 2. **Question:** How does changing the lobe width affect the spectral bandwidth?
> 3. **Question:** How does adding more lobes improve the shape of the slice profile? What is the trade-off regarding the overall pulse width?
> 4. **Question:** What happens when apodization is too large? Pick a value that looks like a good compromise.
> -------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
from custom_RF import CustomRF

class SincRF(CustomRF):
    
    def setup(self):
        super().setup()
        self.title = "Sinc RF Pulse Design"
        self.workspace = WORKSPACE
        
        self.inputs = {
            'lobe_width': pn.widgets.FloatInput(name="Lobe Width (μs)", start=50, end=400, step=50, value=100, width=80),
            'n_lobe': pn.widgets.FloatInput(name="Lobes", start=0, end=10, step=1, value=3, width=80),
            'apodization': pn.widgets.FloatInput(name="Apodization", start=0, end=100, step=1, value=0, width=80)
        }
        
        # auto frequency range based on max bandwidth
        BW_max = 1e-6*self.inputs['lobe_width'].start
        self.plots['spectrum'].fig.x_range.start = -4/BW_max
        self.plots['spectrum'].fig.x_range.end = 4/BW_max
        
        # update plots when any value is changed
        self.inputs['lobe_width'].param.watch(self.pulse_settings_handler, 'value_throttled')
        self.inputs['n_lobe'].param.watch(self.pulse_settings_handler, 'value_throttled')
        self.inputs['apodization'].param.watch(self.pulse_settings_handler, 'value_throttled')

        # manually trigger for first update
        self.inputs['lobe_width'].param.trigger('value_throttled')
    
    
    def calc_pulse_shape(self):
        width = max(1, self.inputs['n_lobe'].value)*self.inputs['lobe_width'].value*1e-6 # max(1, ...) to prevent width=0 when n_lobes=0
        N = int(width / 1e-6)
        if N > 1000:
            N = 1000
        dt = width / N
        pts = np.sinc(np.linspace(-self.inputs['n_lobe'].value, self.inputs['n_lobe'].value,N))
        pts*=np.exp(-self.inputs['apodization'].value*(np.linspace(-1, 1, N)**2))
        return width, pts
        

lab3exp4 = SincRF(state_id='exp4')
lab3exp4()

> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Task 3.2: 1D Slice Profile**
> With the Sinc pulse designed, we can now use it to create a 1D image of the slice profile.
> You will need Equation 1: $$\Delta z = \frac{\Delta F}{\bar\gamma \cdot G_{ss}}$$ where $\bar\gamma = 42.58 \times 10^{6} \ \mathrm{Hz}/\mathrm{T}$ for the hydrogen nucleus.
> 1. **Question:** Based on the bandwidth ($\Delta$F) of the Sinc pulse you have designed and a slice select gradient $G_{ss}=100 \ \mathrm{mT}/\mathrm{m}$, calculate the expected thickness of the slice profile.
> 2. Insert the **Shim Sample (#001)** (make sure to centre the sample using the depth gauge), set the slice select gradient, and run the experiment.
> 3. **Question:** How does the width of the slice profile compare to your predicted width?
> 4. Try changing the flip angle to 30 and 180 and compare the resulting spectrum.
> 5. **Question:** At what flip angle (out of 30, 90, or 180) does the signal spectrum look most similar to the RF pulse spectrum shown above?
> -------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
class Lab3Task5SpinEchoExperiment(BaseExperiment):
    
    def setup(self):
        self.title = "Sinc RF Pulse Slice Profile Measurement"
        self.workspace = WORKSPACE
        self.seq = Sequence('programs/custom_pulse_SE.py')
        self.plots = {
            'signal': SignalPlot(),
            'image': Image1DPlot()
        }
        
        with open(os.path.join(GLOBALS_DIR, 'gradient_calibration.yaml'), 'r') as f:
            data = yaml.load(f, Loader=yaml.SafeLoader)
            self.G_CAL = 1/(GAMMA_BAR*data['gradient_calibration']) # convert
            self.log.debug(f'G_CAL: {self.G_CAL}')
        
        self.inputs={
            'GSS':pn.widgets.FloatInput(name="Slice Select Gradient (mT/m)", start=0, end=150, step=10, value=100, width=200, disabled=False),
            'flip_angle':pn.widgets.FloatInput(name="Target Flip Angle (degrees)", start=10, end=360, step=10, value=90, width=200, disabled=False)
        }
        self.DEC = 4
        self.G_IMAGE = 0.1 # T/m
    
    def update_par(self):
        self.seq.setpar(
            a_90=self.inputs['flip_angle'].value/90*estimate_a_90(lab3exp4.pulse_shape,lab3exp4.pulse_width),
            t_90=lab3exp4.pulse_width,
            shape_90=lab3exp4.pulse_shape,
            g_slice=(0,0,-1e-3*self.inputs['GSS'].value/self.G_CAL),
            g_read=(0,0,self.G_IMAGE/self.G_CAL),
            n_scans=4,
            n_samples=200*self.DEC,
            t_dw=20e-6/self.DEC, # using a long dwell time for narrow bandwith to more easily see the spectrum shape
            t_end=0.5
        )
    
    async def update_plots(self, *args, **kwargs):
        await self.seq.fetch_data()
        self.plots['signal'].update(seqdata=self.seq.data, t_dw=self.seq.par.t_dw)
        self.plots['image'].update(
            seqdata=self.seq.data,
            t_dw=self.seq.par.t_dw,
            g_read_mag=self.G_CAL*np.linalg.norm(self.seq.par.g_read),
            dec=self.DEC
        )
            

lab3exp5 = Lab3Task5SpinEchoExperiment(state_id='exp5')
lab3exp5()

### Note: Phase Rewinding Gradients

There may be some twisting in the imaginary components of the spectrum in the previous experiment. This is due to dephasing. The application of the slice-select gradient during the soft pulse causes the spins to accumulate different amounts of phase based on their position along the the gradient's axis. This phase dispersion of the transverse magnetization will result in a loss of signal when averaged together. Since the dephasing that occurs is linear, the phase effects can be cancelled by applying a gradient in the opposite direction. This is done using a rephasing gradient. In general, the area of the rephasing gradient is half that of the slice-select gradient. This is done under the assumption that the majority of the spins are tipped into the transverse plane at the centre of the 90-degree pulse and, therefore, phase dispersion primarily occurs during the second half of the slice select gradient. While this assumption is a good starting point, the size of the refocusing gradient needs to be calculated based on the soft pulse design if the phase effects are to be cancelled perfectly.

<center><img src="Images/refocusing_gradient.png" width="800"></center>
<center><figcaption style="width: 600px;">Figure 3.3: Slice Select Spin Echo pulse sequence without (left) and with (right) rephasing lobes</figcaption></center>

> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Task 3.3: 2D RARE Using Sinc Shaped Soft Pulse**
> 1. Insert **Phantom #005**, and run the experiment below with the Z-Y axes selected.
> 2. Adjust the slice select gradient to change the slice thickness until only the middle cell is visible. You may need to also modify the sinc pulse design above if the gradient is reaching the maximum allowed. The sample can also be moved up or down if it is not centred.
> 3. Change the axes to X-Y and run again. You should see only the middle cell appearing in the image. If not, go back and adjust the soft pulse/gradient parameters until you do.
> 4. **Save/screenshot the image**
> -------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# Experiment 6: 2D RARE experiment with slice gradient and sinc slice pulse

class Task6RARE2DExperiment(Base2DRAREExperiment):
    def setup(self):
        super().setup() # call parent class setup method
        
        self.seq = Sequence('programs/custom_pulse_RARE.py')
        
        self.title = "2D Image with Sinc RF Pulse and Slice Select Gradient"
        
        self.inputs = {
            'axis_f': pn.widgets.RadioButtonGroup(name='Freq Enc Axis', options=['X', 'Y', 'Z'], value='X', width = 200),
            'axis_p': pn.widgets.RadioButtonGroup(name='Phase Enc Axis', options=['X', 'Y', 'Z'], value='Y', width = 200),
            'GSS': pn.widgets.FloatInput(name="Slice Select Gradient (mT/m)", start=0, end=150, step=10, value=100, width=200, disabled=False)
        }

    def layout_inputs(self):
        return pn.Column(
            self.inputs['GSS'],
            pn.Row(pn.pane.Markdown('### Frequency Encoding Axis', width=200), self.inputs['axis_f']),
            pn.Row(pn.pane.Markdown('### Phase Encoding Axis', width=200), self.inputs['axis_p']),
            sizing_mode='stretch_width'
        )
    
    def g_phase_mag(self, fov, gamma_bar=GAMMA_BAR):
        """ Returns phase gradient required for a given FOV """
        dt = self.seq.par.t_phase/(len(self.seq.par.g_phase_2)//2)
        return max(0, min(1, 1/(self.G_CAL*fov*gamma_bar*dt))) # clip to range 0 to 1
    
    def fov_phase(self, gamma_bar=GAMMA_BAR):
        """ Returns phase encode direction FOV """
        dt = self.seq.par.t_phase/(len(self.seq.par.g_phase_2)//2)
        g = self.G_CAL*np.max(np.linalg.norm(self.seq.par.g_phase_2, axis=1))
        return 1/(g*gamma_bar*dt)

    def update_par(self):
        self.seq.setpar(
            a_90=estimate_a_90(lab3exp4.pulse_shape,lab3exp4.pulse_width),
            t_90=lab3exp4.pulse_width,
            shape_90=lab3exp4.pulse_shape,
            g_slice=(0,0,-1e-3*self.inputs['GSS'].value/self.G_CAL),
            t_phase=320e-6,
            g_phase_2=np.zeros((RESOLUTION, 3)),
            n_ETL=RESOLUTION,
            n_scans=16,
            n_samples=RESOLUTION*self.DEC,
            t_dw=20e-6/self.DEC,
            t_end=1,
            # t_grad_ramp=50e-6, # TODO: fix strange artifacts with larger t_grad_ramp
            # n_grad_ramp=5
        )
        
        # set calculated pars
        self.seq.setpar(
            n_ETL=len(self.seq.par.g_phase_2),
            t_read=self.seq.par.t_dw*self.seq.par.n_samples
        )
        self.seq.setpar(
            t_phase=self.seq.par.t_read/2
        )
        
        g_phase_prop = np.flip(np.linspace(1, -1, RESOLUTION, endpoint=False))
        self.seq.setpar(
            g_read = self.g_read_mag(self.FOV)*AXIS_VECTORS[self.inputs['axis_f'].value],
            g_phase_2 = np.outer(self.g_phase_mag(self.FOV)*g_phase_prop, AXIS_VECTORS[self.inputs['axis_p'].value])
        )

lab3exp6 = Task6RARE2DExperiment(state_id='exp6')
lab3exp6()

## 4. SLR Soft Pulse Design

The Sinc method of designing a slice select RF pulse works well at small flip angles, but is not ideal for 90 degree pulses and is quite poor for 180 pulses. A more optimal method, that is widely used, is the Shinnar-Le Roux algorithm. This algorithm takes into account the non-linearity of NMR excitation to achieve equal ripple in the final magnetization spectrum and, therefore, signal spectrum.

For 90 degree pulses, this method results in a RF pulse spectrum with minimal sidelobes at the expense of poor flatness on top. However, since the signal level resulting from a near 90 degree flip angle pulse is not very sensitive to the exact flip angle, the slice profile is still quite flat.

The SLR algorithm is also used to design asymmetrical RF pulses which are used for advanced techniques. 

> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Task 4.1: Pulse Design For a Target Slice Thickness Using SLR**
> You will need Equation 1: $$\Delta z = \frac{\Delta F}{\bar\gamma \cdot G_{ss}}$$ where $\bar\gamma = 42.58 \times 10^{6} \ \mathrm{Hz}/\mathrm{T}$ for the hydrogen nucleus.
> 1. **Question**: Calculate the required bandwidth to achieve a 1 mm slice with the maximum slice gradient, $G_{ss}=150 \ \mathrm{mT}/\mathrm{m}$, and enter the value below (take care with units).
> 2. Insert the **Shim Sample**.
> 3. Measure the slice profile with the above slice select gradient and check the thickness is correct. You may increase the pulse duration to improve the sharpness of the edges of the slice.
> -------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# Experiment 7

from matipo.util.pulseshape import calc_soft_pulse

class SlrRF(CustomRF):
    
    def setup(self):
        super().setup()
        self.title = "SLR RF Pulse Design"
        self.workspace = WORKSPACE
        
        self.inputs = {
            'width': pn.widgets.FloatInput(name="Duration (µs)", start=100, end=1000, step=10, value=300, width=80),
            'bandwidth': pn.widgets.FloatInput(name="Bandwidth (kHz)", start=4, end=40, step=1, value=20, width=80)
        }
        
        self.plots['spectrum'].fig.x_range.start = -2*1e3* self.inputs['bandwidth'].end
        self.plots['spectrum'].fig.x_range.end = 2*1e3* self.inputs['bandwidth'].end
        
        # update plots when any value is changed
        self.inputs['width'].param.watch(self.pulse_settings_handler, 'value_throttled')
        self.inputs['bandwidth'].param.watch(self.pulse_settings_handler, 'value_throttled')
        
        # manually trigger for first update
        self.inputs['width'].param.trigger('value_throttled')
    
    
    def calc_pulse_shape(self):
        # if pulse width*bandwidth is too small, force pulse width to increase
        if 1e-6*self.inputs['width'].value*1e3*self.inputs['bandwidth'].value < 1.44:
            self.inputs['width'].value = 1e3*1.44/self.inputs['bandwidth'].value
        width = 1e-6*self.inputs['width'].value
        bw = 1e3*self.inputs['bandwidth'].value
        N, dt, pts = calc_soft_pulse(width, bw)
        return width, pts
        

lab3exp7 = SlrRF(state_id='exp7')
lab3exp7()

In [ ]:
class Lab3Task8SpinEchoExperiment(BaseExperiment):
    
    def setup(self):
        self.title = "SLR RF Pulse Slice Profile Measurement"
        self.workspace = WORKSPACE
        self.seq = Sequence('programs/custom_pulse_SE.py')
        self.plots = {
            'signal': SignalPlot(),
            'image': Image1DPlot()
        }
        
        with open(os.path.join(GLOBALS_DIR, 'gradient_calibration.yaml'), 'r') as f:
            data = yaml.load(f, Loader=yaml.SafeLoader)
            self.G_CAL = 1/(GAMMA_BAR*data['gradient_calibration']) # convert
            self.log.debug(f'G_CAL: {self.G_CAL}')
        
        self.inputs = {
            'GSS': pn.widgets.FloatInput(name="Slice Select Gradient (mT/m)", start=0, end=150, step=10, value=150, width=200, disabled=False)
        }
 
        self.DEC = 4
        self.G_IMAGE = 0.1 # T/m
    
    def update_par(self):
        self.seq.setpar(
            a_90=min(1, abs(AREA_90/(np.mean(lab3exp7.pulse_shape)*lab3exp7.pulse_width))),
            t_90=lab3exp7.pulse_width,
            shape_90=lab3exp7.pulse_shape,
            g_slice=(0,0,-1e-3*self.inputs['GSS'].value/self.G_CAL),
            g_read=(0,0,self.G_IMAGE/self.G_CAL),
            n_scans=4,
            n_samples=200*self.DEC,
            t_dw=20e-6/self.DEC, # using a long dwell time for narrow bandwith to more easily see the spectrum shape
            t_end=0.5
        )
    
    async def update_plots(self, *args, **kwargs):
        await self.seq.fetch_data()
        self.plots['signal'].update(seqdata=self.seq.data, t_dw=self.seq.par.t_dw)
        self.plots['image'].update(
            seqdata=self.seq.data,
            t_dw=self.seq.par.t_dw,
            g_read_mag=self.G_CAL*np.linalg.norm(self.seq.par.g_read),
            dec=self.DEC
        )
            

lab3exp8 = Lab3Task8SpinEchoExperiment(state_id='exp8')
lab3exp8()

## 5. Controlling Slice Position

> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Task 5.1: Locating the Hidden Text**
>1. Insert **Phantom #006** and adjust the settings to get an XY image of a slice through the text inside the phantom.
>2. **Hint:** Try reducing the slice select gradient to get a thicker slice and viewing a Z-Y projection to identify the plane where the text is. You can measure the Z offset of a bright plane and calculate the required frequency offset to position the slice correctly. The text is located in one of the bright planes that are visible in a Z-Y projection.
> 3. **Save/screenshot the image**
> -------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# Experiment 9: 2D RARE experiment with slice gradient and SLR slice pulse

class Task9RARE2DExperiment(Base2DRAREExperiment):
    def setup(self):
        super().setup() # call parent class setup method
        
        self.seq = Sequence('programs/custom_pulse_RARE.py')
        
        self.title = "2D Image with SLR RF Pulse and Slice Select Gradient"
        
        self.inputs = {
            'axis_f': pn.widgets.RadioButtonGroup(name='Freq Enc Axis', options=['X', 'Y', 'Z'], value='X', width = 200),
            'axis_p': pn.widgets.RadioButtonGroup(name='Phase Enc Axis', options=['X', 'Y', 'Z'], value='Y', width = 200),
            'GSS': pn.widgets.FloatInput(name="Slice Select Gradient (mT/m)", start=0, end=150, step=10, value=100, width=200, disabled=False),
            'freq_offset': pn.widgets.FloatInput(name="Pulse Freq. Offset (kHz)", start=-30, end=30, step=1, value=0, width=200)
        }

    def layout_inputs(self):
        return pn.Column(
            self.inputs['GSS'],
            self.inputs['freq_offset'],
            pn.Row(pn.pane.Markdown('### Frequency Encoding Axis', width=200), self.inputs['axis_f']),
            pn.Row(pn.pane.Markdown('### Phase Encoding Axis', width=200), self.inputs['axis_p']),
            sizing_mode='stretch_width'
        )
    
    def g_phase_mag(self, fov, gamma_bar=GAMMA_BAR):
        """ Returns phase gradient required for a given FOV """
        dt = self.seq.par.t_phase/(len(self.seq.par.g_phase_2)//2)
        return max(0, min(1, 1/(self.G_CAL*fov*gamma_bar*dt))) # clip to range 0 to 1
    
    def fov_phase(self, gamma_bar=GAMMA_BAR):
        """ Returns phase encode direction FOV """
        dt = self.seq.par.t_phase/(len(self.seq.par.g_phase_2)//2)
        g = self.G_CAL*np.max(np.linalg.norm(self.seq.par.g_phase_2, axis=1))
        return 1/(g*gamma_bar*dt)

    def update_par(self):
        self.seq.setpar(
            a_90=min(1, abs(AREA_90/(np.mean(lab3exp7.pulse_shape)*lab3exp7.pulse_width))),
            t_90=lab3exp7.pulse_width,
            shape_90=lab3exp7.pulse_shape,
            f_slice_offset=1e3*self.inputs['freq_offset'].value,
            g_slice=(0,0,1e-3*self.inputs['GSS'].value/self.G_CAL),
            g_phase_2=np.zeros((RESOLUTION, 3)),
            n_scans=64,
            n_samples=RESOLUTION*self.DEC,
            t_dw=20e-6/self.DEC,
            t_end=1,
        )
        
        # set calculated pars
        self.seq.setpar(
            n_ETL=len(self.seq.par.g_phase_2),
            t_read=self.seq.par.t_dw*self.seq.par.n_samples
        )
        self.seq.setpar(
            t_phase=self.seq.par.t_read/2
        )
        
        g_phase_prop = np.flip(np.linspace(1, -1, RESOLUTION, endpoint=False))
        self.seq.setpar(
            g_read = self.g_read_mag(self.FOV)*AXIS_VECTORS[self.inputs['axis_f'].value],
            g_phase_2 = np.outer(self.g_phase_mag(self.FOV)*g_phase_prop, AXIS_VECTORS[self.inputs['axis_p'].value])
        )
        

lab3exp9 = Task9RARE2DExperiment(state_id='exp9')
lab3exp9()